One time run code:

In [14]:
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

In [15]:
convert("./data/train-images-idx3-ubyte", "./data/train-labels-idx1-ubyte",
        "data/mnist_train.csv", 60000)

In [16]:
convert("./data/t10k-images-idx3-ubyte", "./data/t10k-labels-idx1-ubyte",
        "./data/mnist_test.csv", 10000)

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import sys

In [2]:
import os

In [3]:
df = pd.read_csv("./data/mnist_train.csv", header=None)

In [4]:
df.shape

(60000, 785)

In [5]:
training = df.as_matrix()

In [6]:
Y_train = training[:, 0].astype('int')

In [7]:
X_train = training[:, 1:].astype('float64')

In [8]:
X_train.shape

(60000, 784)

In [9]:
df = pd.read_csv("./data/mnist_test.csv", header=None)

In [10]:
test = df.as_matrix()

In [11]:
test.shape

(10000, 785)

In [12]:
Y_test = test[:, 0].astype('int')

In [13]:
X_test = test[:, 1:].astype('float64')

In [14]:
np.min(X_test)

0.0

In [15]:
np.max(X_test)

255.0

In [16]:
X_test/=255

In [17]:
X_train/=255

In [18]:
np.max(X_test)

1.0

In [19]:
Y_train = pd.get_dummies(Y_train).as_matrix()

In [20]:
Y_test = pd.get_dummies(Y_test).as_matrix()

In [21]:
def ReLu(x, derivative=False):
    if(derivative==False):
        return x*(x > 0)
    else:
        return 1*(x > 0)

In [22]:
def Softmax(x):
    x -= np.max(x)
    sm = (np.exp(x).T / np.sum(np.exp(x),axis=1)).T
    return sm

In [23]:
def CreateWeights(wl1, wl2):
    ##Initialization of the Weights and the Biases with the random gaussian function with mean zeron, and variance between 1/sqtr(num_inputs_layer)
    
    ninputs = 784
    nclass = 10 ##Numer of the class, in this case it is the number of the digits.
    
    #layer1
    w1 = np.random.normal(0, ninputs**-0.5, [ninputs,wl1])
    b1 = np.random.normal(0, ninputs**-0.5, [1,wl1])
    
    #Layer2
    w2 = np.random.normal(0, wl1**-0.5, [wl1,wl2])
    b2 = np.random.normal(0, wl1**-0.5, [1,wl2])

    #Layer3
    w3 = np.random.normal(0, wl2**-0.5, [wl2,nclass])
    b3 = np.random.normal(0, wl2**-0.5, [1,nclass])
    
    return [w1,w2,w3,b1,b2,b3]

In [24]:
def Dropout(x, keep_prop):
    mask = np.random.binomial([np.ones_like(x)],(1-keep_prop))[0]  / (1-keep_prop)
    return x*mask

In [25]:
def predict(weights, x, keep_prop=0):
    
    w1,w2,w3,b1,b2,b3  = weights 
    
    #1-Hidden Layer
    first = ReLu(x@w1+b1)
    first = Dropout(first, keep_prop)

    #2-Hidden Layer
    second = ReLu(first@w2+b2)
    second = Dropout(second, keep_prop)
    
    #Output Layer
    return [first, second, Softmax(second@w3+b3)]

In [26]:
def accuracy(output, y):
    hit = 0
    output = np.argmax(output, axis=1)
    y = np.argmax(y, axis=1)
    for y in zip(output, y):
        if(y[0]==y[1]):
            hit += 1

    p = (hit*100)/output.shape[0]
    return p

In [27]:
def log2(x):
    if(x!=0):
        return np.log(x)
    else:
        return -np.inf
    
def log(y):
    return [[log2(nx) for nx in x]for x in y]

def cost(Y_predict, Y_right):
    
    Loss = -np.mean((np.nan_to_num(Y_right*log(Y_predict)) + np.nan_to_num((1-Y_right)*log(1-Y_predict))),keepdims=True)
    return Loss

In [28]:
x_train = X_train

In [29]:
x_valid = X_test

In [30]:
d_train = Y_train

In [31]:
d_valid = Y_test

In [32]:
def ADAM(weights, x, t, outputs, eta, beta1, beta2, eps, i, nabla, cache=None):
    
    w1,w2,w3,b1,b2,b3  = weights
    
    
    if(cache==None):
            vw1 = np.zeros_like(w1)
            mw1 = np.zeros_like(w1)
            
            vw2 = np.zeros_like(w2)
            mw2 = np.zeros_like(w2)
            
            vw3 = np.zeros_like(w3)
            mw3 = np.zeros_like(w3)
            
            vb1 = np.zeros_like(b1)
            mb1 = np.zeros_like(b1)
            
            vb2 = np.zeros_like(b2)
            mb2 = np.zeros_like(b2)
            
            vb3 = np.zeros_like(b3)
            mb3 = np.zeros_like(b3)
    else:
        vw1,mw1,vw2,mw2,vw3,mw3,vb1,mb1,vb2,mb2,vb3,mb3 = cache
    
    first, second, y = outputs
   
    w3_delta = (t-y)/x.shape[0]
    
    w2_error = w3_delta@w3.T

    w2_delta = w2_error * ReLu(second,derivative=True)

    w1_error = w2_delta@w2.T
    w1_delta = w1_error * ReLu(first,derivative=True)
    
    
    dw3 = (second.T@w3_delta + nabla*w3)
    mw3 = beta1*mw3 + (1-beta1)*dw3
    mt = (mw3) / (1-beta1**i)
    vw3 = beta2*vw3 + (1-beta2)*(dw3**2)
    vt = (vw3) / (1-beta2**i)
    w3 += eta * mt/(np.sqrt(vt) + eps)
    
    db3 = (w3_delta.sum(axis=0)+ nabla*b3)
    mb3 = beta1*mb3 + (1-beta1)*db3
    mt = (mb3) / (1-beta1**i)
    vb3 = beta2*vb3 + (1-beta2)*(db3**2)
    vt = (vb3) / (1-beta2**i)
    b3 += eta * mt/(np.sqrt(vt) + eps)
    
    dw2 = (first.T@w2_delta + nabla*w2)
    mw2 = beta1*mw2 + (1-beta1)*dw2
    mt = (mw2) / (1-beta1**i)
    vw2 = beta2*vw2 + (1-beta2)*(dw2**2)
    vt = (vw2) / (1-beta2**i)
    w2 += eta * mt/(np.sqrt(vt) + eps)
    
    db2 = (w2_delta.sum(axis=0) + nabla*b2)
    mb2 = beta1*mb2 + (1-beta1)*db2
    mt = (mb2) / (1-beta1**i)
    vb2 = beta2*vb2 + (1-beta2)*(db2**2)
    vt = (vb2) / (1-beta2**i)
    b2 += eta * mt/(np.sqrt(vt) + eps)
    
    dw1 = (x.T@w1_delta + nabla*w1)
    mw1 = beta1*mw1 + (1-beta1)*dw1
    mt = (mw1) / (1-beta1**i)
    vw1 = beta2*vw1 + (1-beta2)*(dw1**2)
    vt = (vw1) / (1-beta2**i)
    w1 += eta * mt/(np.sqrt(vt) + eps)
    
    db1 = (w1_delta.sum(axis=0) + nabla*b1)
    mb1 = beta1*mb1 + (1-beta1)*db1
    mt = (mb1) / (1-beta1**i)
    vb1 = beta2*vb1 + (1-beta2)*(db1**2)
    vt = (vb1) / (1-beta2**i)
    b1 += eta * mt/(np.sqrt(vt) + eps)
    
    
    weights = [w1,w2,w3,b1,b2,b3]
    cache = [vw1,mw1,vw2,mw2,vw3,mw3,vb1,mb1,vb2,mb2,vb3,mb3]
    
    return weights, cache

In [33]:
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

def elastic_transform(image, alpha, sigma, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))

    return map_coordinates(image, indices, order=1).reshape(shape)

In [34]:
def run(weights, x_train, y_train, x_valid, y_valid, epochs = 10, nbatchs=25, alpha = 1e-3, decay = 0, beta1=0.9, beta2=0.999, eps=1e-8, l2 = 0.001, keep_prop = 0):
    
    pross = x_train.shape[0]*0.05
    
    history = [[],[]]
    
    index = np.arange(x_train.shape[0])
    cache = None
    print("Train data: %d" % (x_train.shape[0]))
    print("Validation data: %d \n" % (x_valid.shape[0]))
    mtime = 0
    inter_adam = 1
    
    for j in range(epochs):
        np.random.shuffle(index)
        t = 0
        iterations = round(x_train.shape[0]/nbatchs)
        prog = ""
        sacurr = 0
        sloss = 0
        sys.stdout.write("\nEpochs: %2d \ %2d \n"% (j+1,epochs))
        stime = 0
        timeIT = time.time()
        for i in range(iterations):
            timeI = time.time()
            f = i*nbatchs
            l = f+nbatchs
            
            if(l>(x_train.shape[0]-1)):
                l = x_train.shape[0]
                
            x = np.array([elastic_transform(xx.reshape(28,28),15,3).reshape(784) for xx in x_train[index[f:l]]])
            y = y_train[index[f:l]]

            outputs = predict(weights, x, keep_prop)
            
            loss = cost(outputs[-1], y)+ l2 * (np.mean(weights[0],keepdims=True)**2+np.mean(weights[1],keepdims=True)**2+np.mean(weights[2],keepdims=True)**2)/3
            
            
            accuracy_t = accuracy(outputs[-1], y)
            
            sacurr += accuracy_t
            sloss += loss
            
            accuracy_train = sacurr/(i+1)
            loss_train = sloss/(i+1)
            
            weights, cache = ADAM(weights, x, y, outputs, alpha, beta1, beta2, eps, inter_adam, l2, cache)
            
            t+= x.shape[0]
            
            qtd = round(t/pross)
            prog = "["
            for p in range(20):
                if(p<qtd-1):
                    prog += "="
                elif(p==qtd-1):
                    prog += ">"
                else:
                    prog += "."
            prog += "]"

            
            stime += time.time()-timeI
            mtime = stime/(i+1)
            mTimeT = mtime * (iterations-i-1)
            
            sys.stdout.write("\r%5d/%5d %s ETA: %3d s - loss: %.4f  acc: %.4f" % (t, x_train.shape[0], prog, mTimeT, loss_train, accuracy_train))
            
            history[0].append([loss_train, accuracy_train])
            inter_adam += 1
        mtime = time.time()-timeIT
        alpha = alpha - (alpha*decay)
        
        outputs = predict(weights, x_valid)
        
        loss_valid = cost(outputs[-1], y_valid)+ l2 * (np.mean(weights[0],keepdims=True)**2+np.mean(weights[1],keepdims=True)**2+np.mean(weights[2],keepdims=True)**2)/3
        accuracy_valid = accuracy(outputs[-1], y_valid)
        
        sys.stdout.write("\r%5d/%5d %s ETA: %3d s loss: %.4f  acc: %.4f - lossValid: %.4f  accValid: %.4f " % ( t, x_train.shape[0], prog, mtime, loss_train, accuracy_train, loss_valid, accuracy_valid))
        history[1].append([loss_valid, accuracy_valid])
        
    return weights, history 

In [35]:
weights = CreateWeights(784, 784)

In [36]:
alpha = 1e-3
epochs = 60
nbatchs = 100
weights, history = run(weights, 
              x_train, d_train, 
              x_valid, d_valid, 
              epochs = epochs,
              nbatchs=nbatchs, 
              alpha = alpha, 
              decay = 0.2, 
              beta1=0.9, 
              beta2=0.999,
              eps=1e-8,
              l2 = 1e-7, 
              keep_prop = 0.1)

Train data: 60000
Validation data: 10000 


Epochs:  1 \ 60 
60000/60000 [===================>] ETA: 130 s loss: 0.0590  acc: 89.0917 - lossValid: 0.0155  accValid: 97.3600 
Epochs:  2 \ 60 
60000/60000 [===================>] ETA: 129 s loss: 0.0317  acc: 94.4483 - lossValid: 0.0122  accValid: 97.8900 
Epochs:  3 \ 60 
60000/60000 [===================>] ETA: 130 s loss: 0.0258  acc: 95.4500 - lossValid: 0.0102  accValid: 98.1200 
Epochs:  4 \ 60 
60000/60000 [===================>] ETA: 130 s loss: 0.0221  acc: 96.0850 - lossValid: 0.0076  accValid: 98.6300 
Epochs:  5 \ 60 
60000/60000 [===================>] ETA: 130 s loss: 0.0196  acc: 96.5300 - lossValid: 0.0072  accValid: 98.7700 
Epochs:  6 \ 60 
60000/60000 [===================>] ETA: 130 s loss: 0.0180  acc: 96.8133 - lossValid: 0.0064  accValid: 98.8300 
Epochs:  7 \ 60 
60000/60000 [===================>] ETA: 129 s loss: 0.0168  acc: 97.0667 - lossValid: 0.0061  accValid: 98.8800 
Epochs:  8 \ 60 
60000/60000 [================

KeyboardInterrupt: 

In [36]:
weights = CreateWeights(2500, 200)

alpha = 1e-3
epochs = 30
nbatchs = 100
weights, history = run(weights, 
              x_train, d_train, 
              x_valid, d_valid, 
              epochs = epochs,
              nbatchs=nbatchs, 
              alpha = alpha, 
              decay = 0.1, 
              beta1=0.9, 
              beta2=0.999,
              eps=1e-8,
              l2 = 1e-7, 
              keep_prop = 0.25)

Train data: 60000
Validation data: 10000 


Epochs:  1 \ 30 
60000/60000 [===================>] ETA: 145 s loss: 0.0619  acc: 88.5083 - lossValid: 0.0162  accValid: 97.0900 
Epochs:  2 \ 30 
60000/60000 [===================>] ETA: 146 s loss: 0.0351  acc: 93.9267 - lossValid: 0.0125  accValid: 97.9000 
Epochs:  3 \ 30 
60000/60000 [===================>] ETA: 143 s loss: 0.0295  acc: 94.7983 - lossValid: 0.0110  accValid: 98.1300 
Epochs:  4 \ 30 
60000/60000 [===================>] ETA: 144 s loss: 0.0254  acc: 95.4283 - lossValid: 0.0091  accValid: 98.5000 
Epochs:  5 \ 30 
60000/60000 [===================>] ETA: 144 s loss: 0.0230  acc: 95.9733 - lossValid: 0.0078  accValid: 98.5800 
Epochs:  6 \ 30 
60000/60000 [===================>] ETA: 144 s loss: 0.0213  acc: 96.2800 - lossValid: 0.0069  accValid: 98.8700 
Epochs:  7 \ 30 
60000/60000 [===================>] ETA: 151 s loss: 0.0197  acc: 96.5483 - lossValid: 0.0072  accValid: 98.8600 
Epochs:  8 \ 30 
34000/60000 [==========>.....

/home/paperspace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in multiply
  if sys.path[0] == '':


60000/60000 [===================>] ETA: 156 s loss: 0.0186  acc: 96.7533 - lossValid: 0.0065  accValid: 98.8400 
Epochs:  9 \ 30 
60000/60000 [===================>] ETA: 162 s loss: 0.0173  acc: 97.0783 - lossValid: 0.0065  accValid: 98.8300 
Epochs: 10 \ 30 
60000/60000 [===================>] ETA: 154 s loss: 0.0165  acc: 97.1467 - lossValid: 0.0066  accValid: 98.8200 
Epochs: 11 \ 30 
60000/60000 [===================>] ETA: 171 s loss: 0.0160  acc: 97.2233 - lossValid: 0.0065  accValid: 98.9000 
Epochs: 12 \ 30 
60000/60000 [===================>] ETA: 173 s loss: 0.0154  acc: 97.3767 - lossValid: 0.0056  accValid: 99.0100 
Epochs: 13 \ 30 
60000/60000 [===================>] ETA: 177 s loss: 0.0148  acc: 97.4100 - lossValid: 0.0058  accValid: 99.0200 
Epochs: 14 \ 30 
60000/60000 [===================>] ETA: 175 s loss: 0.0140  acc: 97.6067 - lossValid: 0.0053  accValid: 99.0000 
Epochs: 15 \ 30 
60000/60000 [===================>] ETA: 172 s loss: 0.0130  acc: 97.7950 - lossValid: 0.00

In [37]:
weights = CreateWeights(2500, 2000)

alpha = 1e-3
epochs = 70
nbatchs = 100
weights, history = run(weights, 
              x_train, d_train, 
              x_valid, d_valid, 
              epochs = epochs,
              nbatchs=nbatchs, 
              alpha = alpha, 
              decay = 0.1, 
              beta1=0.9, 
              beta2=0.999,
              eps=1e-8,
              l2 = 1e-7, 
              keep_prop = 0.25)

Train data: 60000
Validation data: 10000 


Epochs:  1 \ 70 
60000/60000 [===================>] ETA: 312 s loss: 0.0571  acc: 89.1450 - lossValid: 0.0150  accValid: 97.4700 
Epochs:  2 \ 70 
60000/60000 [===================>] ETA: 306 s loss: 0.0345  acc: 93.9033 - lossValid: 0.0116  accValid: 97.8700 
Epochs:  3 \ 70 
60000/60000 [===================>] ETA: 349 s loss: 0.0277  acc: 95.1833 - lossValid: 0.0115  accValid: 97.9800 
Epochs:  4 \ 70 
60000/60000 [===================>] ETA: 358 s loss: 0.0255  acc: 95.5917 - lossValid: 0.0082  accValid: 98.5800 
Epochs:  5 \ 70 
60000/60000 [===================>] ETA: 317 s loss: 0.0224  acc: 96.0333 - lossValid: 0.0078  accValid: 98.7100 
Epochs:  6 \ 70 
14100/60000 [====>...............] ETA: 237 s - loss: 0.0201  acc: 96.4539

/home/paperspace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in multiply
  if sys.path[0] == '':


60000/60000 [===================>] ETA: 305 s loss: 0.0205  acc: 96.4233 - lossValid: 0.0085  accValid: 98.5700 
Epochs:  7 \ 70 
60000/60000 [===================>] ETA: 307 s loss: 0.0190  acc: 96.7250 - lossValid: 0.0064  accValid: 98.9300 
Epochs:  8 \ 70 
60000/60000 [===================>] ETA: 307 s loss: 0.0176  acc: 96.8733 - lossValid: 0.0059  accValid: 98.9700 
Epochs:  9 \ 70 
60000/60000 [===================>] ETA: 299 s loss: 0.0166  acc: 97.0767 - lossValid: 0.0065  accValid: 98.9900 
Epochs: 10 \ 70 
60000/60000 [===================>] ETA: 305 s loss: 0.0162  acc: 97.1967 - lossValid: 0.0058  accValid: 98.9300 
Epochs: 11 \ 70 
60000/60000 [===================>] ETA: 308 s loss: 0.0148  acc: 97.5133 - lossValid: 0.0054  accValid: 99.1000 
Epochs: 12 \ 70 
60000/60000 [===================>] ETA: 308 s loss: 0.0142  acc: 97.5150 - lossValid: 0.0051  accValid: 99.1600 
Epochs: 13 \ 70 
60000/60000 [===================>] ETA: 302 s loss: 0.0131  acc: 97.7117 - lossValid: 0.00

In [37]:
weights = CreateWeights(200, 2000)

alpha = 1e-3
epochs = 30
nbatchs = 100
weights, history = run(weights, 
              x_train, d_train, 
              x_valid, d_valid, 
              epochs = epochs,
              nbatchs=nbatchs, 
              alpha = alpha, 
              decay = 0.2, 
              beta1=0.9, 
              beta2=0.999,
              eps=1e-8,
              l2 = 1e-7, 
              keep_prop = 0.4)

Train data: 60000
Validation data: 10000 


Epochs:  1 \ 30 
60000/60000 [===================>] ETA:  97 s loss: 0.0868  acc: 83.2267 - lossValid: 0.0214  accValid: 96.1200 
Epochs:  2 \ 30 
60000/60000 [===================>] ETA:  97 s loss: 0.0532  acc: 90.3567 - lossValid: 0.0153  accValid: 97.4000 
Epochs:  3 \ 30 
60000/60000 [===================>] ETA:  99 s loss: 0.0453  acc: 91.8533 - lossValid: 0.0139  accValid: 97.7100 
Epochs:  4 \ 30 
60000/60000 [===================>] ETA:  98 s loss: 0.0411  acc: 92.7233 - lossValid: 0.0128  accValid: 97.8100 
Epochs:  5 \ 30 
60000/60000 [===================>] ETA: 100 s loss: 0.0380  acc: 93.3433 - lossValid: 0.0114  accValid: 98.1200 
Epochs:  6 \ 30 
60000/60000 [===================>] ETA:  98 s loss: 0.0357  acc: 93.5233 - lossValid: 0.0115  accValid: 98.0500 
Epochs:  7 \ 30 
60000/60000 [===================>] ETA:  98 s loss: 0.0346  acc: 93.8500 - lossValid: 0.0104  accValid: 98.2100 
Epochs:  8 \ 30 
60000/60000 [================

In [38]:
weights = CreateWeights(3000, 100)

alpha = 1e-3
epochs = 30
nbatchs = 100
weights, history = run(weights, 
              x_train, d_train, 
              x_valid, d_valid, 
              epochs = epochs,
              nbatchs=nbatchs, 
              alpha = alpha, 
              decay = 0.2, 
              beta1=0.9, 
              beta2=0.999,
              eps=1e-8,
              l2 = 1e-7, 
              keep_prop = 0.1)

Train data: 60000
Validation data: 10000 


Epochs:  1 \ 30 
60000/60000 [===================>] ETA: 180 s loss: 0.0599  acc: 88.8117 - lossValid: 0.0148  accValid: 97.2400 
Epochs:  2 \ 30 
60000/60000 [===================>] ETA: 185 s loss: 0.0315  acc: 94.5650 - lossValid: 0.0108  accValid: 98.1000 
Epochs:  3 \ 30 
60000/60000 [===================>] ETA: 178 s loss: 0.0249  acc: 95.6000 - lossValid: 0.0087  accValid: 98.4000 
Epochs:  4 \ 30 
60000/60000 [===================>] ETA: 174 s loss: 0.0211  acc: 96.2983 - lossValid: 0.0077  accValid: 98.6200 
Epochs:  5 \ 30 
60000/60000 [===================>] ETA: 176 s loss: 0.0191  acc: 96.6700 - lossValid: 0.0075  accValid: 98.6800 
Epochs:  6 \ 30 
60000/60000 [===================>] ETA: 181 s loss: 0.0178  acc: 96.8650 - lossValid: 0.0066  accValid: 98.8300 
Epochs:  7 \ 30 
60000/60000 [===================>] ETA: 172 s loss: 0.0161  acc: 97.2600 - lossValid: 0.0072  accValid: 98.7100 
Epochs:  8 \ 30 
60000/60000 [================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



60000/60000 [===================>] ETA: 216 s loss: 0.0120  acc: 97.9333 - lossValid: 0.0050  accValid: 99.0600 
Epochs: 15 \ 30 
60000/60000 [===================>] ETA: 215 s loss: 0.0118  acc: 98.0167 - lossValid: 0.0049  accValid: 99.1400 
Epochs: 16 \ 30 
60000/60000 [===================>] ETA: 211 s loss: 0.0114  acc: 97.9883 - lossValid: 0.0048  accValid: 99.1200 
Epochs: 17 \ 30 
24100/60000 [=======>............] ETA: 130 s - loss: 0.0120  acc: 97.9004

/home/paperspace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in multiply
  if sys.path[0] == '':


60000/60000 [===================>] ETA: 239 s loss: 0.0115  acc: 97.9883 - lossValid: 0.0049  accValid: 99.1200 
Epochs: 18 \ 30 
60000/60000 [===================>] ETA: 270 s loss: 0.0117  acc: 98.0167 - lossValid: 0.0047  accValid: 99.1600 
Epochs: 19 \ 30 
  100/60000 [....................] ETA: 239 s - loss: 0.0172  acc: 97.0000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



60000/60000 [===================>] ETA: 269 s loss: 0.0113  acc: 98.0850 - lossValid: 0.0046  accValid: 99.1800 
Epochs: 23 \ 30 
60000/60000 [===================>] ETA: 210 s loss: 0.0109  acc: 98.1050 - lossValid: 0.0046  accValid: 99.1800 
Epochs: 24 \ 30 
60000/60000 [===================>] ETA: 231 s loss: 0.0108  acc: 98.1833 - lossValid: 0.0047  accValid: 99.1600 
Epochs: 25 \ 30 
60000/60000 [===================>] ETA: 215 s loss: 0.0107  acc: 98.1650 - lossValid: 0.0046  accValid: 99.1600 
Epochs: 26 \ 30 
60000/60000 [===================>] ETA: 213 s loss: 0.0110  acc: 98.0917 - lossValid: 0.0046  accValid: 99.1400 
Epochs: 27 \ 30 
60000/60000 [===================>] ETA: 207 s loss: 0.0108  acc: 98.1100 - lossValid: 0.0046  accValid: 99.1800 
Epochs: 28 \ 30 
60000/60000 [===================>] ETA: 211 s loss: 0.0111  acc: 98.0800 - lossValid: 0.0047  accValid: 99.1500 
Epochs: 29 \ 30 
60000/60000 [===================>] ETA: 202 s loss: 0.0106  acc: 98.1800 - lossValid: 0.00

In [40]:
def SGD(weights, x, t, outputs, eta, gamma, nabla, cache=None):
    
    w1,w2,w3,b1,b2,b3  = weights
    
    
    if(cache==None):
            vw1 = np.zeros_like(w1)
            vw2 = np.zeros_like(w2)
            vw3 = np.zeros_like(w3)
            vb1 = np.zeros_like(b1)
            vb2 = np.zeros_like(b2)
            vb3 = np.zeros_like(b3)
    else:
        vw1,vw2,vw3,vb1,vb2,vb3 = cache
    
    first, second, y = outputs
   
    w3_delta = (t-y)
    
    w2_error = w3_delta@w3.T

    w2_delta = w2_error * ReLu(second,derivative=True)

    w1_error = w2_delta@w2.T
    w1_delta = w1_error * ReLu(first,derivative=True)
    
    eta = -eta/x.shape[0]
    nabla *= 2;
    
    vw3 = gamma*vw3 + eta * (second.T@w3_delta + nabla*w3)
    vb3 = gamma*vb3 + eta * w3_delta.sum(axis=0)

    vw2 = gamma*vw2 + eta * (first.T@w2_delta + nabla*w2)
    vb2 = gamma*vb2 + eta * w2_delta.sum(axis=0)

    vw1 = gamma*vw1 + eta * (x.T@w1_delta + nabla*w1)
    vb1 = gamma*vb1 + eta * w1_delta.sum(axis=0)
    
    
    w3 -= vw3
    b3 -= vb3

    w2 -= vw2
    b2 -= vb2

    w1 -= vw1
    b1 -= vb1
    
    weights = [w1,w2,w3,b1,b2,b3]
    cache = [vw1,vw2,vw3,vb1,vb2,vb3]
    
    return weights, cache

In [41]:
def run_sgd(weights, x_train, y_train, x_valid, y_valid, epochs = 10, nbatchs=25, alpha = 1e-3, decay = 0, momentum = 0, l2 = 0.001, keep_prop = 0):
    
    pross = x_train.shape[0]*0.05
    
    history = [[],[]]
    
    index = np.arange(x_train.shape[0])
    cache = None
    print("Train data: %d" % (x_train.shape[0]))
    print("Validation data: %d \n" % (x_valid.shape[0]))
    mtime = 0
    
    for j in range(epochs):
        np.random.shuffle(index)
        t = 0
        iterations = round(x_train.shape[0]/nbatchs)
        prog = ""
        sacurr = 0
        sloss = 0
        sys.stdout.write("\nEpochs: %2d \ %2d \n"% (j+1,epochs))
        stime = 0
        timeIT = time.time()
        for i in range(iterations):
            timeI = time.time()
            f = i*nbatchs
            l = f+nbatchs
            
            if(l>(x_train.shape[0]-1)):
                l = x_train.shape[0]
                
            x = np.array([elastic_transform(xx.reshape(28,28),15,3).reshape(784) for xx in x_train[index[f:l]]])
            y = y_train[index[f:l]]

            outputs = predict(weights, x, keep_prop)
            
            loss = cost(outputs[-1], y)
            
            
            accuracy_t = accuracy(outputs[-1], y)
            
            sacurr += accuracy_t
            sloss += loss
            
            accuracy_train = sacurr/(i+1)
            loss_train = sloss/(i+1)
            
            weights, cache = SGD(weights, x, y, outputs, alpha, momentum, l2, cache)
            
            t+= x.shape[0]
            
            qtd = round(t/pross)
            prog = "["
            for p in range(20):
                if(p<qtd-1):
                    prog += "="
                elif(p==qtd-1):
                    prog += ">"
                else:
                    prog += "."
            prog += "]"

            
            stime += time.time()-timeI
            mtime = stime/(i+1)
            mTimeT = mtime * (iterations-i-1)
            
            sys.stdout.write("\r%5d/%5d %s ETA: %3d s - loss: %.4f  acc: %.4f" % (t, x_train.shape[0], prog, mTimeT, loss_train, accuracy_train))
            
            history[0].append([loss_train, accuracy_train])
        mtime = time.time()-timeIT
        alpha = alpha - (alpha*decay)
        
        outputs = predict(weights, x_valid)
        
        loss_valid = cost(outputs[-1], y_valid)
        accuracy_valid = accuracy(outputs[-1], y_valid)
        
        sys.stdout.write("\r%5d/%5d %s ETA: %3d s loss: %.4f  acc: %.4f - lossValid: %.4f  accValid: %.4f " % ( t, x_train.shape[0], prog, mtime, loss_train, accuracy_train, loss_valid, accuracy_valid))
        history[1].append([loss_valid, accuracy_valid])
        
    return weights, history

In [ ]:
weights = CreateWeights(784,784)

In [44]:
alpha = 1e-2
epochs = 20
nbatchs = 100
weights, history = run_sgd(weights, 
              x_train, d_train, 
              x_valid, d_valid, 
              epochs = epochs,
              nbatchs=nbatchs, 
              alpha = alpha, 
              decay = alpha/(epochs*2), 
              momentum = 0.9, 
              l2 = 1e-3, 
              keep_prop = 0.25)

Train data: 60000
Validation data: 10000 


Epochs:  1 \ 20 
60000/60000 [===================>] ETA: 117 s loss: 0.0505  acc: 91.1683 - lossValid: 0.0186  accValid: 96.9400 
Epochs:  2 \ 20 
60000/60000 [===================>] ETA: 116 s loss: 0.0436  acc: 92.2300 - lossValid: 0.0156  accValid: 97.2900 
Epochs:  3 \ 20 
60000/60000 [===================>] ETA: 116 s loss: 0.0395  acc: 92.9950 - lossValid: 0.0136  accValid: 97.6000 
Epochs:  4 \ 20 
60000/60000 [===================>] ETA: 116 s loss: 0.0357  acc: 93.7383 - lossValid: 0.0122  accValid: 97.9700 
Epochs:  5 \ 20 
60000/60000 [===================>] ETA: 117 s loss: 0.0330  acc: 94.1817 - lossValid: 0.0110  accValid: 98.1600 
Epochs:  6 \ 20 
60000/60000 [===================>] ETA: 116 s loss: 0.0317  acc: 94.3900 - lossValid: 0.0105  accValid: 98.1900 
Epochs:  7 \ 20 
60000/60000 [===================>] ETA: 115 s loss: 0.0303  acc: 94.7117 - lossValid: 0.0101  accValid: 98.2400 
Epochs:  8 \ 20 
60000/60000 [================